In [1]:
from Impute import fill_with_et
import pandas as pd
import numpy as np

In [2]:
from hyperimpute.plugins.imputers import Imputers, ImputerPlugin

imputers = Imputers()

# stdlib
import copy
from time import time, strftime, localtime
from typing import Any
import warnings

# third party
from IPython.display import display
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from pydantic import validate_arguments
from scipy.stats import wasserstein_distance
from sklearn.preprocessing import MinMaxScaler

# hyperimpute absolute
import hyperimpute.logger as log
from hyperimpute.plugins.imputers import Imputers
from hyperimpute.plugins.utils.metrics import RMSE
from hyperimpute.plugins.utils.simulate import simulate_nan
from hyperimpute.utils.distributions import enable_reproducible_results
from hyperimpute.utils.metrics import generate_score, print_score

enable_reproducible_results()

warnings.filterwarnings("ignore")


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def ampute(
        x: pd.DataFrame,
        mechanism: str,
        p_miss: float,
        column_limit: int = 8,
        sample_columns: bool = True,
) -> tuple:
    columns = x.columns
    column_limit = min(len(columns), column_limit)

    if sample_columns:
        sampled_columns = columns[
            np.random.choice(len(columns), size=column_limit, replace=False)
        ]
    else:
        sampled_columns = columns[list(range(column_limit))]

    x_simulated = simulate_nan(
        x[sampled_columns].values, p_miss, mechanism, sample_columns=sample_columns
    )

    isolated_mask = pd.DataFrame(x_simulated["mask"], columns=sampled_columns)
    isolated_x_miss = pd.DataFrame(x_simulated["X_incomp"], columns=sampled_columns)

    mask = pd.DataFrame(np.zeros(x.shape), columns=columns)
    mask[sampled_columns] = pd.DataFrame(isolated_mask, columns=sampled_columns)

    x_miss = pd.DataFrame(x.copy(), columns=columns)
    x_miss[sampled_columns] = isolated_x_miss

    return (
        pd.DataFrame(x, columns=columns),
        x_miss,
        mask,
    )


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def scale_data(X: pd.DataFrame) -> pd.DataFrame:
    preproc = MinMaxScaler()
    cols = X.columns
    return pd.DataFrame(preproc.fit_transform(X), columns=cols)


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def simulate_scenarios(
        X: pd.DataFrame, column_limit: int = 8, sample_columns: bool = True, percentages: list = [0.1, 0.3, 0.5],
) -> pd.DataFrame:
    X = scale_data(X)

    datasets: dict = {}

    mechanisms = ["MAR", "MNAR", "MCAR"]
    # percentages = [0.1, 0.3, 0.5, 0.7, 0.9]

    for ampute_mechanism in mechanisms:
        for p_miss in percentages:
            if ampute_mechanism not in datasets:
                datasets[ampute_mechanism] = {}

            datasets[ampute_mechanism][p_miss] = ampute(
                X,
                ampute_mechanism,
                p_miss,
                column_limit=column_limit,
                sample_columns=sample_columns,
            )

    return datasets


def ws_score(imputed: pd.DataFrame, ground: pd.DataFrame) -> pd.DataFrame:
    res = 0
    for col in range(ground.shape[1]):
        res += wasserstein_distance(
            np.asarray(ground)[:, col], np.asarray(imputed)[:, col]
        )
    return res


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def benchmark_model(
        name: str,
        model: Any,
        X: pd.DataFrame,
        X_miss: pd.DataFrame,
        mask: pd.DataFrame,
) -> tuple:
    start = time()

    imputed = model.fit_transform(X_miss.copy())

    distribution_score = ws_score(imputed, X)
    rmse_score = RMSE(np.asarray(imputed), np.asarray(X), np.asarray(mask))

    log.info(f"benchmark {model.name()} took {time() - start}")
    return rmse_score, distribution_score


def benchmark_standard(
        model_name: str,
        X: pd.DataFrame,
        X_miss: pd.DataFrame,
        mask: pd.DataFrame,
) -> tuple:
    imputer = imputers.get(model_name)
    return benchmark_model(model_name, imputer, X, X_miss, mask)


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def evaluate_dataset(
        name: str,
        evaluated_model: Any,
        X_raw: pd.DataFrame,
        ref_methods: list = ["mean", "missforest", "ice", "gain", "sinkhorn", "softimpute"],
        scenarios: list = ["MAR", "MCAR", "MNAR"],
        miss_pct: list = [0.1, 0.3, 0.5],
        sample_columns: bool = True,
) -> tuple:
    imputation_scenarios = simulate_scenarios(X_raw, sample_columns=sample_columns, percentages=miss_pct)

    rmse_results: dict = {}
    distr_results: dict = {}

    for scenario in scenarios:

        rmse_results[scenario] = {}
        distr_results[scenario] = {}

        for missingness in miss_pct:

            log.debug(f"  > eval {scenario} {missingness}")
            rmse_results[scenario][missingness] = {}
            distr_results[scenario][missingness] = {}

            try:
                x, x_miss, mask = imputation_scenarios[scenario][missingness]
                (our_rmse_score, our_distribution_score) = benchmark_model(
                    name, copy.deepcopy(evaluated_model), x, x_miss, mask
                )
                rmse_results[scenario][missingness]["our"] = our_rmse_score
                distr_results[scenario][missingness]["our"] = our_distribution_score

                for method in ref_methods:
                    x, x_miss, mask = imputation_scenarios[scenario][missingness]

                    (
                        mse_score,
                        distribution_score,
                    ) = benchmark_standard(method, x, x_miss, mask)
                    rmse_results[scenario][missingness][method] = mse_score
                    distr_results[scenario][missingness][method] = distribution_score
            except BaseException as e:
                print(f"scenario failed {str(e)}")

                continue
    return rmse_results, distr_results


def compare_models(
        name: str,
        evaluated_model: Any,
        X_raw: pd.DataFrame,
        ref_methods: list = ["mean", "missforest", "ice", "gain", "sinkhorn", "softimpute"],
        scenarios: list = ["MNAR"],
        miss_pct: list = [0.1, 0.3, 0.5, 0.7],
        n_iter: int = 2,
        sample_columns: bool = True,
        display_results: bool = True,
        n_jobs: int = 1,
) -> dict:
    dispatcher = Parallel(n_jobs=n_jobs)
    start = time()

    def add_metrics(
            store: dict, scenario: str, missingness: float, method: str, score: float
    ) -> None:
        if scenario not in store:
            store[scenario] = {}
        if missingness not in store[scenario]:
            store[scenario][missingness] = {}
        if method not in store[scenario][missingness]:
            store[scenario][missingness][method] = []

        store[scenario][missingness][method].append(score)

    rmse_results_dict: dict = {}
    distr_results_dict: dict = {}

    def eval_local(it: int) -> Any:
        enable_reproducible_results(it)
        log.debug(f"> evaluation trial {it}")
        return evaluate_dataset(
            name=name,
            evaluated_model=evaluated_model,
            X_raw=X_raw,
            ref_methods=ref_methods,
            scenarios=scenarios,
            miss_pct=miss_pct,
            sample_columns=sample_columns,
        )

    repeated_evals_results = dispatcher(delayed(eval_local)(it) for it in range(n_iter))

    for (
            local_rmse_results,
            local_distr_results,
    ) in repeated_evals_results:
        for scenario in local_rmse_results:
            for missingness in local_rmse_results[scenario]:
                for method in local_rmse_results[scenario][missingness]:
                    add_metrics(
                        rmse_results_dict,
                        scenario,
                        missingness,
                        method,
                        local_rmse_results[scenario][missingness][method],
                    )
                    add_metrics(
                        distr_results_dict,
                        scenario,
                        missingness,
                        method,
                        local_distr_results[scenario][missingness][method],
                    )

    rmse_results = []
    distr_results = []

    rmse_str_results = []
    distr_str_results = []

    for scenario in rmse_results_dict:

        for missingness in rmse_results_dict[scenario]:

            local_rmse_str_results = [scenario, missingness]
            local_distr_str_results = [scenario, missingness]

            local_rmse_results = [scenario, missingness]
            local_distr_results = [scenario, missingness]

            for method in ["our"] + ref_methods:
                rmse_mean, rmse_std = generate_score(
                    rmse_results_dict[scenario][missingness][method]
                )
                rmse_str = print_score((rmse_mean, rmse_std))
                distr_mean, distr_std = generate_score(
                    distr_results_dict[scenario][missingness][method]
                )
                distr_str = print_score((distr_mean, distr_std))

                local_rmse_str_results.append(rmse_str)
                local_rmse_results.append((rmse_mean, rmse_std))

                local_distr_str_results.append(distr_str)
                local_distr_results.append((distr_mean, distr_std))

            rmse_str_results.append(local_rmse_str_results)
            rmse_results.append(local_rmse_results)
            distr_str_results.append(local_distr_str_results)
            distr_results.append(local_distr_results)

    if display_results:
        log.info(f"benchmark took {time() - start}")
        headers = (
                ["Scenario", "miss_pct [0, 1]"]
                + [f"Evaluated: {evaluated_model.name()}"]
                + ref_methods
        )

        sep = "\n==========================================================\n\n"
        cur_time = strftime("%Y%m%d_%H%M%S", localtime())
        data = pd.DataFrame(rmse_str_results, columns=headers)
        data.to_csv(f"./{name}_rmse.csv", index=False)
        display(data)

        print(sep + "Wasserstein score")

        data = pd.DataFrame(distr_str_results, columns=headers)
        data.to_csv(f"./{name}_dis.csv", index=False)
        display(data)

    return {
        "headers": headers,
        "rmse": rmse_results,
        "wasserstein": distr_results,
    }

In [3]:
class EtImputer(ImputerPlugin):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._model = fill_with_et

    @staticmethod
    def name():
        return "et"

    @staticmethod
    def hyperparameter_space():
        return []

    def _fit(self, *args, **kwargs):
        return self

    def _transform(self, df):
        # 按照缺失值的比例进行排序
        miss_rate = df.isnull().sum() / df.shape[0]
        cols = miss_rate.sort_values().index.tolist()
        cols = [col for col in cols if miss_rate[col] > 0]
        for col in cols:
            df_col_filled = self._model(df, col)
            df[col] = df_col_filled[col]
        return df


imputers.add("et", EtImputer)

In [4]:
# class RidgeImputer(ImputerPlugin):
#     def __init__(self, *args, **kwargs):
#         super().__init__(*args, **kwargs)
#         self._model = fill_with_ridge
# 
#     @staticmethod
#     def name():
#         return "Ridge"
# 
#     @staticmethod
#     def hyperparameter_space():
#         return []
# 
#     def _fit(self, *args, **kwargs):
#         return self
# 
#     def _transform(self, df):
#         # 按照缺失值的比例进行排序
#         miss_rate = df.isnull().sum() / df.shape[0]
#         cols = miss_rate.sort_values().index.tolist()
#         cols = [col for col in cols if miss_rate[col] > 0]
#         for col in cols:
#             df_col_filled = self._model(df, col)
#             df[col] = df_col_filled[col]
#         return df
# 
# 
# imputers.add("Ridge", RidgeImputer)

In [4]:
from pathlib import Path

datasets = [Path("./dataset/Crystal_structure.csv")]
datasets

[WindowsPath('dataset/Crystal_structure.csv')]

In [6]:
from tqdm import tqdm

imputer = imputers.get("et")

for d in tqdm(datasets):
    print("Current imputer", imputer.name())
    if Path.exists(Path(f"./Et-knn-{d.stem}_rmse.csv")):
        print(f"skip {d.stem}")
        continue
    df = pd.read_csv(d)
    df = df.select_dtypes(include=[np.number])
    print(d.stem)
    compare_models(
        name=f"Et-knn-{d.stem}",
        evaluated_model=imputer,
        X_raw=df,
        ref_methods=["mean", "hyperimpute", "missforest", "gain", "sinkhorn"],
        scenarios=["MAR", "MCAR", "MNAR"],
        miss_pct=[0.1, 0.2, 0.3, 0.4, 0.5],
        n_iter=5
    )

  0%|          | 0/1 [00:00<?, ?it/s]

Current imputer et
Crystal_structure
scenario failed Expected more than 1 value per channel when training, got input size torch.Size([1, 100])
scenario failed Expected more than 1 value per channel when training, got input size torch.Size([1, 100])
scenario failed Expected more than 1 value per channel when training, got input size torch.Size([1, 100])


,Scenario,"miss_pct [0, 1]",Evaluated: et,mean,hyperimpute,missforest,gain,sinkhorn
0,MAR,0.1,0.0254 +/- 0.0102,0.2421 +/- 0.0264,0.0462 +/- 0.0247,0.1036 +/- 0.0278,0.1614 +/- 0.0399,0.1235 +/- 0.0474
1,MAR,0.2,0.0335 +/- 0.0076,0.2412 +/- 0.0348,0.0475 +/- 0.0148,0.1198 +/- 0.0224,0.1499 +/- 0.0374,0.1259 +/- 0.0398
2,MAR,0.3,0.0266 +/- 0.0083,0.2286 +/- 0.0192,0.0901 +/- 0.026,0.1147 +/- 0.0201,0.154 +/- 0.0311,0.1186 +/- 0.034
3,MAR,0.4,0.0335 +/- 0.0134,0.2246 +/- 0.0272,0.0891 +/- 0.0342,0.1071 +/- 0.0197,0.1278 +/- 0.0289,0.1087 +/- 0.0393
4,MAR,0.5,0.0544 +/- 0.0065,0.2297 +/- 0.0137,0.1049 +/- 0.0231,0.1275 +/- 0.0145,0.1663 +/- 0.0213,0.1511 +/- 0.0188
5,MCAR,0.1,0.0196 +/- 0.0049,0.2078 +/- 0.0123,0.0286 +/- 0.0173,0.0946 +/- 0.0125,0.1211 +/- 0.0248,0.0894 +/- 0.0283
6,MCAR,0.2,0.0318 +/- 0.0084,0.2108 +/- 0.0051,0.052 +/- 0.014,0.1059 +/- 0.0104,0.1469 +/- 0.0174,0.1156 +/- 0.0224
7,MCAR,0.3,0.042 +/- 0.0052,0.2167 +/- 0.011,0.1072 +/- 0.0056,0.1135 +/- 0.0082,0.1527 +/- 0.0143,0.1295 +/- 0.0132
8,MCAR,0.4,0.057 +/- 0.0188,0.2154 +/- 0.0079,0.1061 +/- 0.0071,0.1163 +/- 0.0072,0.1506 +/- 0.0138,0.1312 +/- 0.0099
9,MCAR,0.5,0.0636 +/- 0.0162,0.2148 +/- 0.0075,0.1082 +/- 0.0146,0.1156 +/- 0.0125,0.1571 +/- 0.0244,0.1283 +/- 0.0257




Wasserstein score


,Scenario,"miss_pct [0, 1]",Evaluated: et,mean,hyperimpute,missforest,gain,sinkhorn
0,MAR,0.1,0.0024 +/- 0.0007,0.0788 +/- 0.0096,0.0067 +/- 0.0038,0.0241 +/- 0.0051,0.0357 +/- 0.0077,0.0147 +/- 0.0036
1,MAR,0.2,0.0075 +/- 0.0022,0.1516 +/- 0.0237,0.0126 +/- 0.005,0.0596 +/- 0.0119,0.0608 +/- 0.0163,0.0327 +/- 0.0108
2,MAR,0.3,0.0101 +/- 0.0037,0.2186 +/- 0.019,0.0467 +/- 0.0146,0.0824 +/- 0.0127,0.1052 +/- 0.0223,0.0455 +/- 0.0101
3,MAR,0.4,0.0138 +/- 0.004,0.2792 +/- 0.0328,0.0829 +/- 0.0347,0.1037 +/- 0.0203,0.1041 +/- 0.026,0.0595 +/- 0.0182
4,MAR,0.5,0.0327 +/- 0.0066,0.3574 +/- 0.0258,0.0966 +/- 0.0351,0.1413 +/- 0.0157,0.1508 +/- 0.0142,0.0988 +/- 0.0182
5,MCAR,0.1,0.0039 +/- 0.0011,0.126 +/- 0.0124,0.0069 +/- 0.0033,0.0437 +/- 0.0062,0.0481 +/- 0.0094,0.0205 +/- 0.0053
6,MCAR,0.2,0.0132 +/- 0.0027,0.2696 +/- 0.0066,0.0264 +/- 0.0093,0.0975 +/- 0.0084,0.128 +/- 0.0159,0.0466 +/- 0.0084
7,MCAR,0.3,0.0238 +/- 0.0043,0.3932 +/- 0.0283,0.1267 +/- 0.0177,0.1537 +/- 0.0109,0.1742 +/- 0.0263,0.0756 +/- 0.0118
8,MCAR,0.4,0.0453 +/- 0.0126,0.536 +/- 0.0289,0.14 +/- 0.0292,0.1989 +/- 0.0122,0.2375 +/- 0.0269,0.1104 +/- 0.0161
9,MCAR,0.5,0.0654 +/- 0.0188,0.6745 +/- 0.0265,0.1793 +/- 0.0156,0.25 +/- 0.0276,0.3283 +/- 0.0738,0.1347 +/- 0.0238


100%|██████████| 1/1 [3:36:52<00:00, 13012.46s/it]


In [7]:
print("=")

=
